In [ ]:
COLAB: bool = False
if COLAB:
  !git clone https://github.com/RubenCid35/6GSmartRRM
  !mv 6GSmartRRM/* /content/
  !pip install -e .
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

In [ ]:
!nvidia-smi -q | grep 'Power Limit' 

In [ ]:

!cd .. && pip install -e .

In [ ]:
%load_ext autoreload
%autoreload 2
!pip install -q wandb matplotlib seaborn 

In [ ]:
# simple data manipulation
import numpy  as np

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lrs
from   torch.utils.data import DataLoader, TensorDataset, random_split
import torch.cuda.amp as amp # For Automatic Mixed Precision

from functools import partial

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from collections import defaultdict

# progress bar
from   tqdm.notebook import tqdm, trange
import wandb

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns

# wheter we are using colab or not
import os
if not COLAB and not os.path.exists('./data/simulations'):
    os.chdir('..')
    print("current path: ", os.getcwd())

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.evaluation import rate as rate
from g6smart.evaluation.utils import get_cdf
from g6smart.evaluation import rate_torch as rate_metrics
from g6smart.proposals  import loss as loss_funcs, rate_cnn, rate_dnn
from g6smart.data import load_data, create_datasets, download_simulations_data
from g6smart.track import setup_wandb, real_time_plot

config = SimConfig(0)
print(config)

## Datasets

In [ ]:
simulation_path, models_path = download_simulations_data(COLAB)
print("simulations data paths:", simulation_path)
print("saved model location  :", models_path)


In [ ]:
csi_data = load_data(simulation_path, n_samples=120_000)
train_dataset, valid_dataset, tests_dataset = create_datasets(
#    csi_data, split_sizes=[130_000, 60_000, 10_000], batch_size=2048, seed=101
    csi_data, split_sizes=[ 70_000, 30_000, 20_000], batch_size= 1024, seed=101
)
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True )
valid_loader = DataLoader(valid_dataset, batch_size=2048, shuffle=True )
tests_loader = DataLoader(tests_dataset, batch_size=2048, shuffle=False)


## FNN

## FNN Approach

In [ ]:
def min_approx(x: torch.Tensor, p: float = 1e8, mu: float = 0.):
    """
    Differentiable Approximation of Minimum Function. This function approximates
    the value of min(x)

      # based on fC https://mathoverflow.net/questions/35191/a-differentiable-approximation-to-the-minimum-function
    """
    return mu - (1 / p) * torch.logsumexp(-p * (x - mu), dim = 1)

def loss_pure_rate(
  config: SimConfig, C: torch.Tensor, A: torch.Tensor,
  mode: str = 'sum', p: int = 1e8, a: float = 0.5
) -> torch.Tensor:
    sinr = rate_metrics.signal_interference_ratio(config, C, A, None)
    mask = torch.sigmoid(10 * (sinr - 0.01))
    rate = torch.sum(torch.log2(1 + sinr) * A * mask, dim = 1)
    rate = rate / torch.sum(A, dim = 1)

    if mode == 'sum':
      loss_rate = torch.sum(rate, dim = 1)
    elif mode == 'min':
      loss_rate = min_approx(rate, p)
    elif mode == 'max':
      loss_rate = torch.sum(rate, dim = 1)
    elif mode == "mean":
      loss_rate = torch.mean(rate, dim = 1)
    elif mode == "hybrid":
      loss_rate = a * torch.mean(rate, dim = 1) + (1 - a) * min_approx(rate, p)
    return - loss_rate

def loss_interference(C: torch.Tensor, A: torch.Tensor):
  losses = A.unsqueeze(-1) * 10 * torch.log10(C + 1e-12) * A.unsqueeze(-2)
  # losses = 10 * torch.log10(losses + 1e-12)
  return torch.sum(losses.flatten(start_dim = 1), dim = 1)


In [ ]:
sample = torch.rand(256, 100000)
mint = torch.max(sample, dim = 1)[0]
for p in range(-2, 10):
  min1 = loss_funcs.min_approx(sample, p = - 10 ** p)
  min2 = min_approx(sample, p = - 10 ** p)
  dis1 = torch.abs(min1 - mint).mean().item()
  dis2 = torch.abs(min2 - mint).mean().item()
  print(f"{10 ** p:3.2e} -> min (lib): {dis1:7.6e}\tmin (new): {dis2:7.6e}")

In [ ]:
class FNNModel(nn.Module):
  def __init__(
      self, N: int, K: int,
      hidden_dim: int = 1024, hidden_layers: int = 4,
      dropout: float = 1e-1, to_matrix: bool = False
  ):
    super().__init__()
    self.N = N
    self.K = K

    self.triu_mask = torch.triu(
        torch.ones(self.N, self.N, dtype = torch.bool),
        diagonal = 1
    )
    self.to_matrix = to_matrix
    if self.to_matrix:
      self.in_size  = self.N * (self.N - 1) // 2
    else:
      self.in_size  = self.N * (self.N - 1) // 2 * self.K

    self.out_size = self.K * self.N

    layers = [ ]
    #nn.BatchNorm1d(self.in_size),
    #nn.BatchNorm2d(self.K), nn.Flatten()

    dims = [self.in_size] + [hidden_dim] * (hidden_layers + 1) + [self.out_size]

    for _in, _out in zip(dims[:-1], dims[1:]):
      layers.append(nn.Linear(_in, _out))
      torch.nn.init.kaiming_normal_(layers[-1].weight, nonlinearity='relu')
      layers.append(nn.ReLU())
      layers.append(nn.BatchNorm1d(_out))
      layers.append(nn.Dropout(dropout))

    self.model = nn.Sequential(*layers[:-3])

  def preprocess(self, H: torch.Tensor) -> torch.Tensor:
    shape = 'x'.join(map(str, H.shape[1:]))
    assert len(H.shape)  == 4 and H.size(1) == self.K and H.size(2) == self.N and H.size(3) == self.N, (
        f"The input tensor needs to match Bx{self.K}x{self.N}x{self.N}, not Bx{shape} (B is the batch size)"
    )

    # take only the upper triangle
    if self.to_matrix:
      H = torch.sum(H, dim = 1)
      H = H[:, self.triu_mask]
    else:
      Hd = torch.diagonal(H, dim1=2, dim2=3).unsqueeze(-1)
      H  = H / Hd
      H  = H[:, :, self.triu_mask] # B x K x (N * (N - 1) // 2)


    # reshape
    H = 10 * torch.log10(H + 1e-12)

    # could this be learned
    mean = torch.mean(H, dim = 1, keepdim=True)
    std  = torch.std( H, dim = 1, keepdim=True)
    H    = (H - mean) / (std + 1e-8)
    return H

  def forward(self, H: torch.Tensor) -> torch.Tensor:
    H = self.preprocess(H)
    x = self.model(H)
    x = x.reshape(-1, self.K, self.N)
    return F.softmax(x, dim = 1)


# CNN + LSTM

In [ ]:
class CLAllocator(nn.Module):
  def __init__(
    self, n_subnetworks: int, n_subbands: int,
    feature_dim: int = 64, hidden_dim: int = 128
  ):
    super().__init__()
    assert n_subbands >= 4, "the model config only works with at least 4 bands"
    assert n_subnetworks >= 4, "the model config only works with at least 4 bands"

    self.N = n_subnetworks
    self.K = n_subbands

    self.feature_dim = feature_dim
    self.hidden_dim  = hidden_dim

    self.encoder = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(), # GELU generally performs well, keep unless direct performance hit
        nn.MaxPool2d(kernel_size=(2, 2)),

        nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 2)),

        nn.Conv2d(64, self.feature_dim, kernel_size=(3, 3), padding=1),
        nn.BatchNorm2d(self.feature_dim),
        nn.ReLU(),

        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
    )

    self.lstm = nn.LSTM(
        input_size    = self.feature_dim,
        hidden_size   = self.hidden_dim,
        num_layers    = 2,
        batch_first   = True,
        bidirectional = True # Keep bidirectional for potential performance unless proven slower
    )

    lstm_output_dim = self.hidden_dim * 2 if self.lstm.bidirectional else self.hidden_dim

    self.allocator = nn.Sequential(
        nn.Linear(lstm_output_dim, self.hidden_dim * 2),
        nn.ReLU(),
        nn.BatchNorm1d(self.hidden_dim * 2),
        nn.Dropout(p = 0.2),

        nn.Linear(self.hidden_dim * 2, self.hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(self.hidden_dim),
        nn.Dropout(p = 0.2),

        nn.Linear(self.hidden_dim, self.K),
        nn.Softmax(dim = 1) # Keep Softmax only if loss_pure_rate expects probabilities, not logits
    )


  def preprocess(self, H: torch.Tensor) -> torch.Tensor:
    Hdb = 10 * torch.log10(H + 1e-12)
    mean, std = -73, -10 # empirical metrics
    return (Hdb - mean) / std


  def forward(self, H: torch.Tensor) -> torch.Tensor:
    batch_size = H.shape[0]
    H = self.preprocess(H)
    H_reshaped_for_encoder = H.permute(0, 2, 1, 3).reshape(batch_size * self.N, self.K, self.N).unsqueeze(1)

    all_encodings = self.encoder(H_reshaped_for_encoder)
    encodings = all_encodings.reshape(batch_size, self.N, self.feature_dim)

    lstm_output, _ = self.lstm(encodings)

    lstm_output_reshaped = lstm_output.reshape(-1, lstm_output.shape[-1])
    all_probs = self.allocator(lstm_output_reshaped)

    probs = all_probs.reshape(batch_size, self.N, self.K)
    probs = probs.permute(0, 2, 1)
    return probs

K, N = 4, 20
sample = torch.rand(2, K, N, N)
model  = CLAllocator(N, K)
model(sample).shape

In [ ]:
class LSTMAllocator(nn.Module):
  def __init__(
    self, n_subnetworks: int, n_subbands: int,
    hidden_dim: int = 128, lstm_layers: int = 3,
  ):
    super().__init__()
    assert n_subbands >= 4, "the model config only works with at least 4 bands"
    assert n_subnetworks >= 4, "the model config only works with at least 4 bands"

    self.N = n_subnetworks
    self.K = n_subbands

    self.hidden_dim  = hidden_dim
    self.feature_dim = self.N * self.K
      
    self.lstm = nn.LSTM(
        input_size    = self.feature_dim,
        hidden_size   = self.hidden_dim,
        num_layers    = lstm_layers,
        batch_first   = True,
        bidirectional = True # Keep bidirectional for potential performance unless proven slower
    )

    lstm_output_dim = self.hidden_dim * 2 if self.lstm.bidirectional else self.hidden_dim

    self.allocator = nn.Sequential(
        nn.Linear(lstm_output_dim, self.hidden_dim * 2),
        nn.ReLU(),
        nn.BatchNorm1d(self.hidden_dim * 2),
        nn.Dropout(p = 0.2),

        nn.Linear(self.hidden_dim * 2, self.hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(self.hidden_dim),
        nn.Dropout(p = 0.2),

        nn.Linear(self.hidden_dim, self.K),
        nn.Softmax(dim = 1) # Keep Softmax only if loss_pure_rate expects probabilities, not logits
    )

  def forward(self, H: torch.Tensor) -> torch.Tensor:
    batch_size = H.size(0)

    network_vector = H.permute(0, 2, 1, 3)
    network_vector = network_vector.flatten(start_dim = 2)
    lstm_output, _ = self.lstm(network_vector)

    lstm_output_reshaped = lstm_output.reshape(-1, lstm_output.shape[-1])
    all_probs = self.allocator(lstm_output_reshaped)

    probs = all_probs.reshape(batch_size, self.N, self.K)
    probs = probs.permute(0, 2, 1)
    return probs

K, N = 4, 20
sample = torch.rand(2, K, N, N)
model  = LSTMAllocator(N, K)
model(sample).shape

In [ ]:
torch.autograd.set_detect_anomaly(True)
print(torch.cuda.memory_summary(device=None, abbreviated=True))

In [ ]:
percentiles = {}

In [ ]:
# Example

del model
model = CLAllocator(20, 4, 256, 512)
model = LSTMAllocator(20, 4, 512, 4)
#model = FNNModel(20, 4, 512, 3, 0.1, True) # .to(device)
#model = rate_cnn.RateConfirmAllocCNNModel(20, 4, 0.1, True)

train_model(model, config, train_loader, valid_loader, )



In [ ]:
import json
model.eval()
total_loss = 0.
total_bin_error = 0.
metrics = defaultdict(lambda : 0)

rates = []
with torch.no_grad():
  for sample in tqdm(tests_loader, desc = "testing: ", unit=" batch", total = len(tests_loader), leave = False):
    sample = sample[0].to(device)
    A = model(sample)        # soft output
    loss = loss_pure_rate(config, sample, A,  'min', p = 1e6).mean()
    # loss = loss_interference(sample, alloc_prob).mean()
    total_loss += loss.item()
    total_bin_error += loss_funcs.binarization_error(A)

    metrics = loss_funcs.update_metrics(metrics, A, sample, None, config, 4)

    A    = torch.argmax(A, dim = 1)
    sinr = rate_metrics.signal_interference_ratio(config, sample, A, None)
    rate = torch.sum(10 * torch.log2(1 + sinr), dim = 1)
    rates.append(rate.cpu().flatten().numpy())

    del sample, A, loss
    torch.cuda.empty_cache()

total_loss = total_loss / len(tests_loader)
total_bin_error = total_bin_error / len(tests_loader)

metrics = { key: val / len(tests_loader) for key, val in metrics.items()}

print("testing run:")
print("testing batches: ", len(tests_loader))
print("test test error: ", total_loss)
print("test test binarization error: ", total_bin_error)
print("bit rate / quality metrics:\n", json.dumps(metrics, indent = 2))

percentiles[model_name] = get_cdf(np.hstack(rates))

In [ ]:
data = np.hstack(rates)
np.percentile(data, 0.05), np.median(data)


In [ ]:
_, ax = plt.subplots(1, 1, figsize = (8, 8))
for name, (pos, per) in percentiles.items():
  ax.plot(pos, per, label = name)

plt.ylim(0, 1)
plt.legend()
plt.show()

In [ ]:
data = {"percentiles": per }
data.update( { 
    f"{name}_values" : pos for name, (pos, _) in percentiles.items() 
})
print(data.columns)
pd.DataFrame(data).to_csv("../results.csv")